<h2>Agentic RAG App: Groq LLM</h2>

<h3>LLM

In [1]:
# LangChain has a Groq chat model integration:
from langchain_groq import ChatGroq

/Users/cameroncochrane/Data Projects/Agentic-RAG/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the API groq API key:
from pathlib import Path
# Bespoke API key loader functions:
from load_keys import *
import os

In [3]:
p = Path("keys/groq.json")
groq_api_key = load_groq_key(p)
os.environ['GROQ_API_KEY'] = groq_api_key

In [4]:
# Initialise the model:
model = ChatGroq(
            model="openai/gpt-oss-120b",
            temperature=0.2,
            max_retries=2,
        )

In [5]:
initial_message = [("human","hello, how are you?")]
answer_1 = model.invoke(initial_message) # .invoke to send a message to the model.

In [6]:
extracted_message = answer_1.content # x.content returns the message for the user.
print(extracted_message)

Hello! I'm doing great, thank you for asking. How can I assist you today?


In [7]:
internet_message = [("human","Do you have internet access?")]
internet_answer = model.invoke(internet_message)
print(internet_answer.content)

I don’t have the ability to browse the web or access real‑time internet content. My responses are based on the information I was trained on, which goes up through September 2021 (with a limited knowledge update through 2024). If you need the most current data or something that requires live lookup, you’ll need to check a reliable source directly.


In [8]:
def load_llm(model_type:str = 'openai/gpt-oss-120b',api_key_path:str = "keys/groq.json"):
    
    p = Path(api_key_path)

    groq_api_key = load_groq_key(p)
    os.environ['GROQ_API_KEY'] = groq_api_key
    
    model = ChatGroq(
            model=model_type,
            temperature=0.2,
            max_retries=2,
        )
    
    return model

In [9]:
def give_message(model,message):

    full_answer = model.invoke(message)
    extracted_answer = full_answer.content

    return extracted_answer

In [10]:
model_1 = load_llm()
extracted_answer = give_message(model_1,"What are some of your use cases?")
print(extracted_answer)

Sure! Here’s a quick overview of the many ways people (and businesses) put a language model like me to work:

| Category | Typical Use Cases | Example Prompts |
|----------|-------------------|-----------------|
| **Content Creation** | • Blog posts, articles, newsletters<br>• Social‑media copy, ad headlines, product descriptions<br>• Scripts for videos, podcasts, webinars | “Write a 600‑word blog post about sustainable travel trends in 2024.” |
| **Creative Writing** | • Short stories, flash fiction, poetry<br>• Plot outlines, character bios, world‑building details<br>• Dialogue polishing, writer’s block busting | “Give me three possible twists for a mystery set in a lunar mining colony.” |
| **Education & Tutoring** | • Explaining concepts (math, science, history, etc.)<br>• Generating practice problems & solutions<br>• Language learning (vocab, grammar, conversation) | “Explain the difference between Newton’s first and second laws with everyday examples.” |
| **Programming & Technic

In [11]:
# We now have the basic llm functionality sorted. Can move onto adding agents with CrewAI

<h4>CrewAI

In [12]:
from crewai import Agent, Task, Crew, Process, LLM
from load_keys import *

In [13]:
openai_key_path = Path('keys/open_ai.json')
openai_key = load_openai_key(openai_key_path)
os.environ['OPENAI_API_KEY'] = openai_key

In [23]:
def build_test_crew(llm):
    researcher = Agent(
        role="Researcher",
        backstory="",
        goal="Retrieve and compile evidence using the provided LLM as the sole knowledge source (do NOT call external/local tools).",
        llm=llm,
        tools=[],
        verbose=False,
    )

    writer = Agent(
        role="Content Writer",
        backstory="",
        goal="Write a structured answer grounded in the evidence produced by the Researcher (LLM).",
        llm=llm,
        tools=[],
        verbose=False,
    )

    critic = Agent(
        role="Reviewer",
        backstory="",
        goal="Check the answer for unsupported claims, missing info, and clarity against the Researcher's evidence.",
        llm=llm,
        tools=[],
        verbose=False,
    )

    t1 = Task(
        description=(
            "Research this query using only the LLM: {query}\nReturn bullet findings." # It's important to add the specific variable containing the query hereQ!!
        ),
        expected_output="A bullet list of findings derived from the LLM only.",
        agent=researcher,
    )

    t2 = Task(
        description=(
            "Write the answer. Use the Researcher's findings as the source."
        ),
        expected_output="A Markdown response mapped to the Researcher's findings.",
        agent=writer,
        context=[t1],
    )

    t3 = Task(
        description=(
            "Critique the answer against the Researcher's response. List issues and provide fix instructions."
        ),
        expected_output="A list of issues found plus concrete fix instructions.",
        agent=critic,
        context=[t1, t2],
    )

    t4 = Task(description=(
            "Consider the issues and fix instructions provided by the critic and write a final answer to be outputted to the user"
        ),
        expected_output="A Final answer, written in markdown, which considers the points raised by the critic",
        agent=writer,
        context=[t1, t2, t3],)

    return Crew(
        agents=[researcher, writer, critic],
        tasks=[t1, t2, t3, t4],
        process=Process.sequential,
        verbose=False,
    )

llm_for_crew = LLM(
    model="groq/openai/gpt-oss-120b",
    api_key=os.environ["GROQ_API_KEY"],
    temperature=0.2,
)

test_crew = build_test_crew(llm=llm_for_crew)

In [ ]:
# Using the crew:
initial_user_query = "How old is the planet?"

result = test_crew.kickoff(inputs={"query": initial_user_query})
print(result)
# The final answer here mentions the critque mainly...

# The Age of the Earth – Evidence‑Based Consensus (≈ 4.54 billion years)

## 1. Current Scientific Estimate  
**The Earth is estimated to be 4.54 Ga old (± 0.05 Ga, i.e., ± 50 million years).**  

---

## 2. Primary Dating Evidence – Iron‑Nickel Meteorites  

| Sample | Dating Method | Age (Ga) | Uncertainty | Why it matters |
|--------|---------------|----------|-------------|----------------|
| **Canyon Diablo (iron‑nickel meteorite)** | **Uranium–lead (U‑Pb)** | **4.56** | **± 0.02** | Because iron‑nickel meteorites have remained chemically closed since condensation, their U‑Pb clocks record the time of the first solid bodies in the solar system, which is taken as the age of the material from which Earth accreted. This makes the age the **most precise** determination of early solar‑system solids (Jacobsen *et al.*, 2008; Amelin *et al.*, 2002). |

---

## 3. Analytical Techniques  
High‑precision ages are obtained with **thermal‑ionisation mass spectrometry (TIMS)** or **inductively

╭────────────────────────── Trace Batch Finalization ──────────────────────────╮
│ ✅ Trace batch finalized with session ID:                                    │
│ 16ad5fd8-fc5d-48e2-8ed5-1f618b156deb                                         │
│                                                                              │
│ 🔗 View here:                                                                │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/16ad5fd8-fc5d-48e │
│ 2-8ed5-1f618b156deb?access_code=TRACE-6b0ee3dbaa                             │
│ 🔑 Access Code: TRACE-6b0ee3dbaa                                             │
╰──────────────────────────────────────────────────────────────────────────────╯


<h4>Combining it with the vectorstore

In [26]:
# Writing the local search tool that the CrewAI will use, giving the tool the loaded vecstore:
from typing import Type, List, Dict, Any
from pydantic import BaseModel, Field
from crewai.tools import BaseTool
from langchain_community.vectorstores import FAISS
from docstore_functions import *

class LocalSearchArgs(BaseModel):
    query: str = Field(..., description="User question")
    k: int = Field(6, ge=1, le=20, description="Top-k chunks")

class LocalFAISSSearchTool(BaseTool):
    name: str = "local_search"
    description: str = "Search the local FAISS vectorstore and return relevant chunks with source metadata."
    args_schema: Type[BaseModel] = LocalSearchArgs

    def __init__(self, store: FAISS):
        super().__init__()
        self.store = store

    def _run(self, query: str, k: int = 6) -> List[Dict[str, Any]]:
        docs = self.store.similarity_search(query, k=k)
        out = []
        for i, d in enumerate(docs, start=1):
            out.append({
                "id": f"L-{i:04d}",
                "text": d.page_content,
                "source": d.metadata.get("source", ""),
                "page": d.metadata.get("page", None),
                "content_hash": d.metadata.get("content_hash", None),
            })

        return out
    
# Loading the store:
store = load_docstore_from_dir()

ImportError: cannot import name 'is_offline_mode' from 'huggingface_hub' (/Users/cameroncochrane/Data Projects/Agentic-RAG/.venv/lib/python3.12/site-packages/huggingface_hub/__init__.py)